In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoProcessor, Pix2StructForConditionalGeneration
import requests
from PIL import Image

model = Pix2StructForConditionalGeneration.from_pretrained("google/matcha-chartqa").to(0)
processor = AutoProcessor.from_pretrained("google/matcha-chartqa")
url = "https://raw.githubusercontent.com/vis-nlp/ChartQA/main/ChartQA%20Dataset/val/png/20294671002019.png"
url = "https://ichef.bbci.co.uk/images/ic/832xn/p09xd6t1.png"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(images=image, text="What is the highest temperature?", return_tensors="pt").to(0)
predictions = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(predictions[0], skip_special_tokens=True))

4pm


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!git clone https://github.com/acolas1/GAP_COLING2022

Cloning into 'GAP_COLING2022'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 50 (delta 11), reused 39 (delta 5), pack-reused 0
Unpacking objects: 100% (50/50), 11.25 MiB | 10.01 MiB/s, done.


In [ ]:
%cd GAP_COLING2022

/content/GAP_COLING2022


In [ ]:
!pip uninstall transformers
!pip install transformers==3.5

  Using cached transformers-3.5.0-py3-none-any.whl (1.3 MB)
  Using cached tokenizers-0.9.3.tar.gz (172 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sentencepiece-0.1.91.tar.gz (500 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install transformers

  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)


In [ ]:
!pip install transformers --user

In [ ]:
!bash infer.sh

2023-07-07 11:03:07.757016: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/GAP_COLING2022/cli_gap.py", line 14, in <module>
    from run_gap import run
  File "/content/GAP_COLING2022/run_gap.py", line 9, in <module>
    from modeling_gap import GAPBartForConditionalGeneration as GAP
  File "/content/GAP_COLING2022/modeling_gap.py", line 16, in <module>
    from transformers.configuration_bart import BartConfig
ModuleNotFoundError: No module named 'transformers.configuration_bart'


In [ ]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch, os, re

torch.hub.download_url_to_file('https://raw.githubusercontent.com/vis-nlp/ChartQA/main/ChartQA%20Dataset/val/png/multi_col_1229.png', 'chart_example_1.png')

model_name = "ahmed-masry/unichart-chartqa-960"
image_path = "/content/chart_example_1.png"
input_prompt = "<chartqa> What is the lowest value in blue bar? <s_answer>"

model = VisionEncoderDecoderModel.from_pretrained(model_name)
processor = DonutProcessor.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

image = Image.open(image_path).convert("RGB")
decoder_input_ids = processor.tokenizer(input_prompt, add_special_tokens=False, return_tensors="pt").input_ids
pixel_values = processor(image, return_tensors="pt").pixel_values

outputs = model.generate(
    pixel_values.to(device),
    decoder_input_ids=decoder_input_ids.to(device),
    max_length=model.decoder.config.max_position_embeddings,
    early_stopping=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    use_cache=True,
    num_beams=4,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
    return_dict_in_generate=True,
)
sequence = processor.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
sequence = sequence.split("<s_answer>")[1].strip()
print(sequence)

100%|██████████| 34.2k/34.2k [00:00<00:00, 7.33MB/s]


6


In [ ]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image
import requests

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print('----------------------------------------')
print('총 파라미터 수 : ', sum(p.numel() for p in model.parameters()))
print('학습가능 파라미터 수 : ', sum(p.numel() for p in model.parameters() if p.requires_grad == True))
print('학습불가 파라미터 수 : ', sum(p.numel() for p in model.parameters() if p.requires_grad == False))
print('----------------------------------------')

max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(requests.get(image_path, stream=True).raw)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds

url = "https://images.mypetlife.co.kr/content/uploads/2022/07/07093143/afra-ramio-cp2HCVzguw4-unsplash-1024x638.jpg"
url = 'https://raw.githubusercontent.com/vis-nlp/ChartQA/main/ChartQA%20Dataset/val/png/20294671002019.png'

predict_step([url])

Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.4.crossattention.masked_bias', 'decoder.transformer.h.2.crossattention.bias', 'decoder.transformer.h.2.crossattention.masked_bias', 'decoder.transformer.h.7.attn.masked_bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.9.crossattention.bias', 'decoder.transformer.h.3.crossattention.masked_bias', 'decoder.transformer.h.6.attn.masked_bias', 'decoder.transformer.h.7.crossattention.masked_bias', 'decoder.transformer.h.8.crossattention.bias', 'decoder.transformer.h.9.attn.masked_bias', 'decoder.transformer.h.0.attn.masked_bias', 'decoder.transformer.h.4.crossattention.bias', 'decoder.transformer.h.6.attn.bias', 'decoder.transformer.h.5.crossattention.masked_bias', 'decoder.transformer.h.7.crossattention.bias', 'decoder.transformer.h.1.attn.masked_bias', 'decoder.transformer.h.1.crossattention.bias', 'decoder.t

----------------------------------------
총 파라미터 수 :  239195904
학습가능 파라미터 수 :  239195904
학습불가 파라미터 수 :  0
----------------------------------------


['a series of photos showing different types of electronic devices']

In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-yv61m6ks
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-yv61m6ks
  Resolved https://github.com/huggingface/transformers to commit 495729427045c7a58e040fa9bf6df81c16f54208
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7286095 sha256=4c4fbbfa927528a588a49ebee4719de2f4036c8c0905bc277f04fb998f138199
  Stored in directory: /tmp/pip-ephem-wheel-cache-j0bs6zk4/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2


In [ ]:
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
import torch
from PIL import Image
import requests

model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-13b")
processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-13b")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print('----------------------------------------')
print('총 파라미터 수 : ', sum(p.numel() for p in model.parameters()))
print('학습가능 파라미터 수 : ', sum(p.numel() for p in model.parameters() if p.requires_grad == True))
print('학습불가 파라미터 수 : ', sum(p.numel() for p in model.parameters() if p.requires_grad == False))
print('----------------------------------------')

url = "https://raw.githubusercontent.com/salesforce/LAVIS/main/docs/_static/Confusing-Pictures.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
prompt = "What is unusual about this image?"
inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

outputs = model.generate(
        **inputs,
        do_sample=False,
        num_beams=5,
        max_length=256,
        min_length=1,
        top_p=0.9,
        repetition_penalty=1.5,
        length_penalty=1.0,
        temperature=1,
)
outputs[outputs == 0] = 2
generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
print(generated_text)

ImportError: ignored

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 918.8 kB/s eta 0:00:00


In [2]:
!npm install openai

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ openai@3.3.0
added 9 packages from 12 contributors and audited 9 packages in 1.623s

1 package is looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [3]:
!curl https://api.openai.com/v1/models \
  -H "Authorization: Bearer "

{
  "object": "list",
  "data": [
    {
      "id": "whisper-1",
      "object": "model",
      "created": 1677532384,
      "owned_by": "openai-internal",
      "permission": [
        {
          "id": "modelperm-KlsZlfft3Gma8pI6A8rTnyjs",
          "object": "model_permission",
          "created": 1683912666,
          "allow_create_engine": false,
          "allow_sampling": true,
          "allow_logprobs": true,
          "allow_search_indices": false,
          "allow_view": true,
          "allow_fine_tuning": false,
          "organization": "*",
          "group": null,
          "is_blocking": false
        }
      ],
      "root": "whisper-1",
      "parent": null
    },
    {
      "id": "babbage",
      "object": "model",
      "created": 1649358449,
      "owned_by": "openai",
      "permission": [
        {
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "object": "model_permission",
          "created": 1669085501,
          "allow_create_engine": fals